## Branch and bound algorithm for feature selection (Oleksii Hrinchuk)

### Problem description

We have a problem of feature selection. Given a set $X_D$ of $D$ features and our task is to select optimal subset $X_d$ of $d$ features from them. We consider some subset $X_d^*$ optimal if it satisfies

$$
J(X_d^*) = \max_{X_d \subset X_D} J(X_d),
$$

where $J$ is a criterion function which should satisfy a monotonicity property

$$
J(X_s) \geq J(X_t),\;\text{if}\;X_s \supseteq X_t.
$$

The monotonicity is not particularly restrictive, as it merely means that subset of features should be not better than any larger set that contatints the subset. For example, we can use some discriminant function or Bhattacharyya distance as $J$.

Feature set $X_D$ has $C_D^d = \frac{D!}{d!(D-d)!}$ subsets of $d$ elements, so exhaustive search is not acceptable in this problem. We decided to use branch and bound algorithm, which avoids exhaustive search and finds the globally best value for any criterion $J$.

### Minimum solution tree

The solution tree is a fourtuple $(N,E,r,L)$:

* N $-$ node set
* E $-$ edge set, $e(n,n_c) \in E$ denotes that node $n_c$ holds all features hold by node $n$ except one discarded
* r $-$ root node with feature set $X_D$
* L $-$ terminal node set which containts $C_D^d$ nodes each of which holds $d$ features

<img src="tree.png">

### Algorithm

The following notation will be used in the algorithm.

* **LIST(i)**: An ordered list of the features enumerated at level $i$
* **POINTER(i)**: The pointer to the element of **LIST(i)** being currently considered. 
* **SUCCESSOR(i,k)**: The number of successors that the $k_{th}$ element in **LIST(i)** can have.
* **AVAIL**: A list of available feature values that **LIST(i)** can assume.

1) **Step 0** (Initialization): Set $B = B_0$, **AVAIL** = $\{1,2,...,D\}$, $i=1$, **LIST(0)** = $\{0\}$, **SUCCESSOR(0,1)** = $d+1$, **POINTER(0)** = 1.

2) **Step 1** (Initialize **LIST(i)**): Set **NODE** = **POINTER(i-1)**. Compute $J_i(x_1,\dots,x_{i-1},k)$ for all $k$ in **AVAIL**. Rank the features in **AVAIL** in the increasing order of $J_i(x_1,\dots,x_{i-1},k)$ and store the smallest $p$ of these in **LIST(i)** in the increasing order (with the first element in **LIST(i)** being the feature in **AVAIL** yielding the smallest $J_i$), where $p$ = **SUCCESSOR(i-1, NODE)**. Set **SUCCESSOR(i,j)** = $p -j + 1$, for $j = 1,\dots,p$. Remove **LIST(i)** from **AVAIL**.

3) **Step 2** (Select new node): If **LIST(i)** is empty, go to **Step 4**. Otherwise, set $x_i = k$ where $k$ is the last element in **LIST(i)**. Set **POINTER(i)** = $j$ where $j$ is the current number of elements in **LIST(i)**. Delete $k$ from **LIST(i)**.

4) **Step 3** (Check bound): If $J_i(x_1,\dots,x_i) < B$, return $x_i$ to **AVAIL** and go to **Step 4**. If level $i=d$, go to **Step 5**. Otherwise, set $i = i+1$ and go to **Step 1**.

5) **Step 4** (Backtrack): Set $i=i-1$. If $i=0$, terminate the algorithm. Otherwise, return $x_i$ to **AVAIL** and go to **Step 2**.

6) **Step 5** (Final level, update bound): Set $B = J_{D-d}(x_1,\dots,x_{D-d})$ and save $(x_1,\dots,x_{D-d})$ as $(x_1^*,\dots,x_{D-d}^*)$. Return $x_{D-d}$ to **AVAIL**. Go to **Step 4**.

## Implementation

### Imports

In [1]:
# basic imports
import csv
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# scikit learn
from sklearn.preprocessing import scale

### Reading data

In [2]:
# reading CSV file
reader = csv.reader(open('../features_2.csv', 'r'), delimiter=',')
data_full = np.array(list(reader))
reader = csv.reader(open('../all_endpoints_with_missing_values_012615.csv', 'r'), delimiter=',')
activity_full = np.array(list(reader))

In [3]:
# feature names
feature_names = data_full[0, 1:]

# names of the proteins
protein_names = data_full[1:, 0]
protein_names1 = activity_full[1:, 0]
print 'Protein names equality check:', np.array_equal(protein_names1, protein_names)

# names of receptors
receptor_names = activity_full[0, 1:]

# Object-Feature matrix (proteins description)
X = data_full[1:, 1:].astype('double')

# Activity matrix
Y = activity_full[1:, 1:].astype('int16')

# Removing constant features
ind = np.var(X, axis = 0) != 0
X = X[:, ind]
feature_names = feature_names[ind]

Protein names equality check: True


In [20]:
s = 0
for y in Y:
    if (y[:3].sum() < 10):
        s += 1

In [21]:
s

4282

In [10]:
(Y == 999).sum()

41470

### Branch and bound class

In [57]:
class BranchAndBound(object):
    
    def __init__(self, X, Y, d, B0):
        """Class for branch and bound feature selection algorithm."""
        
        # Load data to class
        self.X = X
        self.Y = Y
        self.D = X.shape[1]
        self.d = self.D - d
        self.B = self.bhatta(X[:,:d])
        
        # Initialization  
        a = []
        for i in xrange(self.D):
            a.append((i, 0))
        self.AVAIL = np.array(a, dtype=[('ind', '<i4'), ('rank', '<f4')])
        
        self.i = 1
        
        self.Z = []
        self.Z_star = [j for j in xrange(d)]
        
        self.LIST = [[] for j in xrange(self.D-self.d)]
        self.LIST[0].append(0)
        
        self.SUCCESSOR = [[] for j in xrange(self.D-self.d)]
        self.SUCCESSOR[0].append(self.d+1)
        
        self.POINTER = np.zeros(self.D-self.d, dtype=np.int)
    
    def bhatta(self, X1):
        
        Mean_X1 = [np.sum(x1) for x1 in X1]
        
        res = max(Mean_X1)
        
        return res
    
    def returnx(self):
        """Return x to AVAIL."""
        
        indices = [self.x] + self.Z[:self.i-1]
        Jx = self.bhatta(X[:, indices])
        self.AVAIL = np.concatenate((self.AVAIL, np.array([(self.x, Jx)], 
                                    dtype=[('ind', '<i4'), ('rank', '<f4')])))
        self.AVAIL.sort(order='rank')
    
    def step1(self):
        """Initialize LIST(i)."""
        
        NODE = self.POINTER[self.i-1]
        for f in self.AVAIL:
            indices = [f['ind']] + self.Z[:self.i-1]
            f['rank'] = self.bhatta(X[:, indices])
        self.AVAIL.sort(order='rank')
        
        p = self.SUCCESSOR[self.i-1][NODE]
        
        self.LIST[self.i] = [self.AVAIL[j][0] for j in xrange(p)]
        self.SUCCESSOR[self.i] = p - np.arange(p)  
        self.AVAIL = self.AVAIL[p:]

        self.step2()
        
    def step2(self):
        """Select the first node to check."""
        
        self.x = self.LIST[self.i].pop()
        
        self.Xs = [f['ind'] for f in self.AVAIL]
        indices = self.Xs + self.Z[:self.i-1] + [self.x]
        
        self.Jc = self.bhatta(X[:, indices])
        
        if (i == self.D - self.d):
            self.step3()
            return 0

        self.step3()
        
    def step3(self):
        """Check bound."""
        
        self.returnx()
        
        if (self.Jc <= self.B):
            self.step5()
            return 0
        
        self.step4()

    def step4(self):
        """Update bound."""
        
        self.B = self.Jc
        self.Z_star = self.Z[:self.i-1] + self.Xs + [self.x]
        
        self.step5()
                
    def step5(self):
        """Handle the next node."""
        
        if not self.LIST[self.i]:
            self.step6()
            return 0        
        
        self.POINTER[self.i] = len(self.LIST[self.i]) - 1
        self.x = self.LIST[self.i].pop()
        
        indices = self.Z[:self.i-1] + [self.x]
        if (self.bhatta(X[:, indices]) <= self.B):
            self.returnx()
            self.step5()
            return 0
        
        if (self.i == self.D - self.d):
            self.returnx()
            self.step4()
            return 0
        
        self.Z.append(self.x)
        self.i = self.i + 1
        
        self.step1()
                  
    def step6(self):
        """Backtrack."""
        
        self.i = self.i - 1
        if (self.i == 0):
            print "TERMINATED"
            return 0
        
        self.returnx()
        
        self.step2()
        
    def solve(self):
        """Launch implemented B&B algorithm to find the best feature subset."""
        
        self.step1()
        
        return self.Z_star

### Experiments

In [58]:
X = np.random.randint(2, size=(10,10))
print X
for i in xrange(10):
    print X[:,i].sum(),

[[0 1 1 1 1 0 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 0]
 [1 1 1 0 0 0 1 0 1 0]
 [1 1 0 1 0 1 1 1 0 0]
 [0 1 0 0 1 0 1 0 0 1]
 [0 1 1 0 0 1 1 1 1 1]
 [0 0 1 1 0 0 0 0 0 1]
 [1 0 1 1 0 1 1 0 0 0]
 [0 1 1 0 1 1 1 1 0 0]
 [0 0 1 1 0 1 0 1 0 0]]
4 6 8 5 3 5 7 4 2 3


In [59]:
bnb.bhatta(X[:,:2])

2

In [60]:
bnb = BranchAndBound(X, Y, 2, 0)
features = bnb.solve()
features

TERMINATED


[0, 1]

In [25]:
# bhattacharyya test
import numpy
import math

h1 = [ 1, 2, 3, 4, 5, 6, 7, 8 ];
h2 = [ 6, 5, 4, 3, 2, 1, 0, 0 ];
h3 = [ 8, 7, 6, 5, 4, 3, 2, 1 ];
h4 = [ 1, 2, 3, 4, 4, 3, 2, 1 ];
h5 = [ 8, 8, 8, 8, 8, 8, 8, 8 ];

h = [ h1, h2, h3, h4, h5 ];

def mean( hist ):
    mean = 0.0;
    for i in hist:
        mean += i;
    mean/= len(hist);
    return mean;

def bhatta ( hist1,  hist2):
    # calculate mean of hist1
    h1_ = mean(hist1);

    # calculate mean of hist2
    h2_ = mean(hist2);

    # calculate score
    score = 0;
    for i in range(8):
        score += math.sqrt( hist1[i] * hist2[i] );
    # print h1_,h2_,score;
    score = math.sqrt( 1 - ( 1 / math.sqrt(h1_*h2_*8*8) ) * score );
    return score;

# generate and output scores
scores = [];
for i in range(len(h)):
    score = [];
    for j in range(len(h)):
        score.append( bhatta(h[i],h[j]) );
    scores.append(score);

for i in scores:
    print i

[0.0, 0.5829473992320814, 0.38838260391018525, 0.24018505569731874, 0.19788811106463064]
[0.5829473992320814, 0.0, 0.21900173667175776, 0.40691958649312965, 0.40534773052584305]
[0.38838260391018525, 0.21900173667175776, 0.0, 0.24018505569731874, 0.19788811106463064]
[0.24018505569731874, 0.40691958649312965, 0.24018505569731874, 0.0, 0.16789959640267496]
[0.19788811106463064, 0.40534773052584305, 0.19788811106463064, 0.16789959640267496, 0.0]
